# Learning Keras

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Data loading & preprocessing

Neural networks don't process raw data, like text files, encoded JPEG image files, or CSV files. They process vectorized & standardized representations.

Images need to be read and decoded into **integer tensors**, then converted to **floating point and normalized** (usually between 0 and 1).

### Data loading

Keras models accept 3 types of inputs: NumPy arrays, TensorFlow `Dataset` objects and Python generators that yield batches of data.

Note: 
If you have a large dataset and you are training on GPU(s), consider using Dataset objects, since they will take care of performance-critical details. e.g. Preprocessing with CPU while GPU is busy. Prefetching data on GPU memory, fully utilizing GPU.

`tf.keras.preprocessing.image_dataset_from_directory` turns image files sorted into calss-specific folders into a labeled dataset of image tensors.

In [ ]:
# main_directory/
# ...class_a/
# ......a_image_1.jpg
# ......a_image_2.jpg
# ...class_b/
# ......b_image_1.jpg
# ......b_image_2.jpg

In [3]:
# Make a test data set. Transfer fits to jpg. Not sure if we can use fits directly. Data in jpg is incredibly small, don't know if there is  any quality loss.
# need only to run once
import sunpy.map
import os
import matplotlib.pyplot as plt
path = r'E:\Researches\2020EUVSolarFlare\Data_test\aia_test'
j = 1
for root, dirs, files in os.walk(path):
    for file in files:
        file = os.path.join(root, file)
        map = sunpy.map.Map(file)
        data = map.data
        plt.imsave('test' + str(j) + '.jpg', data, cmap='gray')
        j += 1
        

In [6]:
# Create a dataset.
data_path = r'E:\Program Files\VSCode\2021_ImagingWithDeepLearning\my_test_data'
dataset = keras.preprocessing.image_dataset_from_directory(
  data_path, batch_size=5, image_size=(4096, 4096))

# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
    print(data.shape)  # (5, 4096, 4096, 3)
    print(data.dtype)  # float32
    print(labels.shape)  # (5,)
    print(labels.dtype)  # int32

Found 10 files belonging to 2 classes.
(5, 4096, 4096, 3)
<dtype: 'float32'>
(5,)
<dtype: 'int32'>
(5, 4096, 4096, 3)
<dtype: 'float32'>
(5,)
<dtype: 'int32'>


The label of a sample is the rank of its folder in alphanumeric order. Naturally, this can also be configured explicitly by passing, e.g. `class_names=['class_a', 'class_b']`, in which cases label 0 will be `class_a` and 1 will be `class_b`.

data的最后一个分量是什么意思？3维数据？

Note: 这里jpg格式不好,它损失很多数据,还得自己写一个加载的函数比较好.

Directly we use:

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((examples, labels))

## Data preprocessing with Keras

Once your data is in the form of string/int/float NumpPy arrays, or a Dataset object (or Python generator) that yields batches of string/int/float tensors, it is time to preprocess the data.

### Using Keras preprocessing layers

In Keras, you do in-model data preprocessing via **preprocessing layers**. For image it means image rescaling, cropping, or image data augmentation. Note that the key advantage of using Keras preprocessing layers is that they can be included directly into your model, either during training or after training, which makes your models portable.

The state of a preprocessing layer is obtained by calling `layer.adapt(data)` on a sample of the training data (or all of it).

#### Example: rescaling & center-cropping images

Both the `Rescaling` layer and the `CenterCrop` layer are stateless, so it isn't necessary to call `adapt()` in this case.

In [9]:
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

for data, labels in dataset:
    training_data = data

    # cropper = CenterCrop(height=150, width=150)
    scaler = Rescaling(scale=1.0 / 255)

    output_data = scaler(training_data)
    print("shape:", output_data.shape)
    print("min:", np.min(output_data))
    print("max:", np.max(output_data))

shape: (5, 4096, 4096, 3)
min: 0.0
max: 1.0
shape: (5, 4096, 4096, 3)
min: 0.0
max: 1.0


## Building models with the Keras Functional API

A "layer" is a simple input-output transformation (such as the scaling & center-cropping transformations above). A "model" is a directed acyclic graph of layers. You can think of a model as a "bigger layer" that encompasses multiple sublayers and that can be trained via exposure to data.

**Functional API** the most common and powerful way to build Keras models.
1. specify the shape(and optionally the dtype) of your inpus. If any dimension of your input can vary, you can specify it as `None`. e.g. `(200, 200, 3)` for fixed shape or `(None, None, 3)` for arbitrary shape.

In [10]:
# Let's say we expect our inputs to be RGB images of arbitrary size
inputs = keras.Input(shape=(4096, 4096, 3))

After defining your input(s), you can chain layer transformations on top of your inputs, until your final output:

In [17]:
from tensorflow.keras import layers

# Center-crop images to ?
x = CenterCrop(height=150, width=150)(inputs)
# Rescale images to [0, 1]
x = Rescaling(scale=1.0 / 255)(x)

# Apply some convolution and pooling layers
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)

# Add a dense classifier on top
num_classes = 10
outputs = layers.Dense(num_classes, activation="softmax")(x)

Once you have defined the directed acyclic graph of layers that turns your input(s) into your outputs, instantiate a `Model` object:

In [20]:
model = keras.Model(inputs=inputs, outputs=outputs)

This model behaves basically like a bigger layer. You can call it on batches of data, like this:

In [21]:
for data, labels in dataset:
    processed_data = model(data)
    print(processed_data.shape)

(5, 10)
(5, 10)


This is a classifier model

In [22]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4096, 4096, 3)]   0         
_________________________________________________________________
center_crop_2 (CenterCrop)   (None, 150, 150, 3)       0         
_________________________________________________________________
rescaling_7 (Rescaling)      (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0   

The Functional API also makes it easy to build models that have multiple inputs (for instance, an *image and its metadata*) or multiple outputs (for instance, predicting the class of the image and the likelihood that a user will click on it). For a deeper dive into what you can do, see our guide to the Functional API.

## Training models with `fit()`

The next step is to train your model on your data. The `Model` class features a built-in training loop, the `fit()` method. It accepts `Dataset` objects, Python generators that yield batches of data, or NumPy arrays.

Before you can call `fit()`, you need to specify an `optimizer` and a `loss function` (we assume you are already familiar with these concepts). This is the `compile()` step:

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())

Loss and optimizer can be specified via their string identifiers (in this case their default constructor argument values are used):

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Once your model is compiled, you can start "fitting" the model to the data. Here's what fitting a model looks like with NumPy data:

In [ ]:
model.fit(numpy_array_of_samples, numpy_array_of_labels,
          batch_size=32, epochs=10)

Besides the data, you have to specify two key parameters: the `batch_size` and the number of epochs (iterations on the data). Here our data will get sliced on batches of 32 samples, and the model will iterate 10 times over the data during training.

In [ ]:
model.fit(dataset_of_samples_and_labels, epochs=10)

Since the data yielded by a dataset is expected to be already batched, you don't need to specify the batch size here.

Let's look at it in practice with a toy example model that learns to classify MNIST digits:

In [3]:
from tensorflow.keras import layers
# Get the data as Numpy arrays
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Build a simple model
inputs = keras.Input(shape=(28, 28))
x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

# Train the model for 1 epoch from Numpy data
batch_size = 64
print("Fit on NumPy data")
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1)

# Train the model for 1 epoch using a dataset
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
print("Fit on Dataset")
history = model.fit(dataset, epochs=1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
rescaling (Rescaling)        (None, 28, 28)            0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_______________________________________________________

The fit() call returns a "history" object which records what happened over the course of training. The `history.history` dict contains per-epoch timeseries of metrics values (here we have only one metric, the loss, and one epoch, so we only get a single scalar):

In [4]:
print(history.history)

{'loss': [0.11748092621564865]}


### Keeping track of performance matrics

#### Monitoring metrics

One can pass a list of metric objects to `compile()`


In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)
history = model.fit(dataset, epochs=1)

#### Passing validation data to `fit()`

One can pass validation data to `fit()` to monitor your validation loss & validation metrics. Validation metrics get reported at the end of each epoch.

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
history = model.fit(dataset, epochs=1, validation_data=val_dataset)

### Using callbacks for checkpointing(and more)

Heavy lifting calculations such as ours will need safety measures. **callbacks**, configured in `fit()`, as an important feature of Keras, are objects that get called by the model at different point during training. In particular at the *beginning and end of each batch, and each epoch*.

''If training goes on for more than a few minutes, it's important to save your model at regular intervals during training. You can then use your saved models to restart training in case your training process crashes (this is important for multi-worker distributed training, since with many workers at least one of them is bound to fail at some point).''



You can use callbacks to periodically save your model. Here's a simple example: a `ModelCheckpoint` callback configured to save the model at the end of every epoch. The filename will include the current epoch.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='path/to/my/model_{epoch}',
        save_freq='epoch')
]
model.fit(dataset, epochs=2, callbacks=callbacks)

You can also use callbacks to do things like periodically changing the learning of your optimizer, streaming metrics to a Slack bot, sending yourself an email notification when training is complete, etc.

### Monitoring training progress with `TensorBoard`

TensorBoard, a web app that can display real-time graphs of your metircs(and more).

To use TensorBoard with `fit()`, simply pass a `keras.callbacks.TensorBoard` callback specifying the directory where to store TensorBoard logs

In [ ]:
callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(dataset, epochs=2, callbacks=callbacks)

You can then launch a TensorBoard instance that you can open in your browser to monitor the logs getting written to this location:

tensorboard --logdir=./logs

What's more, you can launch an in-line TensorBoard tab when training models in Jupyter / Colab notebooks.

## After `fit()`: evaluating test performance & generating predictions on new data

Once you have a trained model, you can evaluate its loss and metrics on new data via `evaluate()`:

In [ ]:
loss, acc = model.evaluate(val_dataset)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

You can also generate NumPy arrays of predictions (the activations of the output layer(s) in the model) via `predict()`:

In [ ]:
predictions = model.predict(val_dataset)
print(predictions.shape)

This is what we would be doing.

## Debugging with `run_eagerly=True`

''With eager execution, the Python code you write is the code that gets executed.'' Use it every time you need to debug inside the `fit()` call.

In [ ]:
model.compile(optimizer='adam', loss='mse', run_eagerly=True)

## Speeding up with multiple GPUs

with `tf.distribute` API. If you have multiple GPUs on your machine, you can train your model on all of them by:

1. Creating a `tf.distribute.MirroredStrategy` object
2. Building & compiling your model inside the strategy's scope
3. Calling `fit()` and `evaluate()` on a dataset as usual

In [ ]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()

# Open a strategy scope.
with strategy.scope():
  # Everything that creates variables should be under the strategy scope.
  # In general this is only model construction & `compile()`.
  model = Model(...)
  model.compile(...)

# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# Test the model on all available devices.
model.evaluate(test_dataset)

## Finding the best model configuration with hyperparameter tuning

To find best hyperparameter. Use `Keras Tuner`.

First, place your model definition in a function, that takes a single `hp` argument. Inside this function, replace any value you want to tune with a call to hyperparameter sampling methods, e.g. `hp.Int()` or `hp.Choice()`:

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(784,))
    x = layers.Dense(
        units=hp.Int('units', min_value=32, max_value=512, step=32),
        activation='relu'))(inputs)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

The function should return a compiled model.

Next, instantiate a *tuner object* specifying your optimization objective and other search parameters:

In [ ]:
import keras_tuner

tuner = keras_tuner.tuners.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=100,
    max_trials=200,
    executions_per_trial=2,
    directory='my_dir')

Finally, start the search with the `search()` method, which takes the same arguments as `Model.fit()`:

In [ ]:
tuner.search(dataset, validation_data=val_dataset)

When search is over, you can retrieve the best model(s):

In [ ]:
models = tuner.get_best_models(num_models=2)

Or print a summary of the results:

In [ ]:
tuner.results_summary()